# Calcium imaging 

Calcium imaging is a technique used to perform optical physiology, i.e., to monitor the lectrical activity of neurons with an optical system. A calcium indictor is either injected or genetically encoded in the cell soma. This indicator is a fluorescent protein that binds to calcium, so that a fluorescence signal is emitted when there is a change in calcium concentration.

Influx or outflux of calcium molecules in cells are related to the generation of spikes, so we can use the change in fluorescence as a proxy for neural activity. Since we can capture fluorescence with a microscope, we can use this technique to directly visualize (a certain kind of) neural activity.

There are two main microscopy techniques for detecting calcium fluorescence: **one-photon** and **two-photon** imaging.

The use of one or the other will depend in the specifc scientific question of the experimenter and the available resources. One-photon technologies are usually protable and can be applied in freely moving animals whilst two-photon microscopy requires a laser set up and thus the animal needs to have its head fixed.

#### My notes on the CaImAn paper

<font color='teal'> **Abstract** </font> 

- Data generated from fluorescent microscopy lacks a reproducible analysis pipeline. This paper proposes one such pipeline - CaImAn.

<font color='teal'> **Introduction** </font> 

- Optical methods based on fluorescence activity are a standard tool to study neural circuitry ove extended periods of time.
- For the current data rates, an automated analysis method that gives a **neuron's location and activity within the imaged Field Of View (FOV)** is required.
- Such analysis method ought to take the follwing processing steps:

    1. **Motion correction** - the FOV at each data frame is registered against a template to correct motion artifacts. The artifacts are a result of the finite scanning rate and existing brain motion.
    2. **Source extraction** - different overlapping sources are extracted ant their signals demixed
    3. **Deconvolution** - the neural activity of each identified source is deconvolved from the dynamics of the calcium indicator.

<font color='teal'> **Related work - Source extraction** </font> 

- For calcium imaging videos, we require source extraction methods that are capable of taking into account both the spatial and temporal dimensions of neural activity.
- One example of such an source extraction method is **constrained nonnegative matrix factorization (CNMF)** and unsupervised method.
- The biggest challenge to these methods is dealing with ever growing size of the datsets. 
- Possible solutions are using HPC or performing dimensionality reduction

<font color='teal'> **Related work - Deconvolution** </font> 

- Deconvolution tackles the problem of predicting spikes from fluorescent traces using either supervised or unsupervised methods.

<font color='teal'> **CaImAn** </font> 

- CaImAn can be used for either one- or two-photon imaging
- CaImAn can be used either after all data has been collected or only at the end.
- CaImAn requires mmoderate computing infrastructure (I guess Windows is too much then ahahah)

<font color='teal'> **Contributions - Methods** </font> 

- `CaImAn Batch` tackles the source extraction problem by employ a `MapReduce` framework allowing the analysis of datasets larger than woulf fit in most computers' RAM
- These improvements are presented in the context of the CNMF algorithm
- `CaImAn Online` introduces new initialisation methods and a CNN based approach for detecting new neurons on streaming data.
- `CaImAn` introduces a novel algorithm, CNMF-E, for one-photon imaging for registration of components across multiple days

<font color='teal'> Contributions - Software </font> 

<font color='teal'> Contributions - Data </font>

<font color='teal'> Paper organization </font> 

<font color='teal'> **METHODS** </font> 

<font color='teal'> **Overview of analysis pipeline** </font> 

- First, the data is processed to remove motion artifacts.
- Then, two active components (neurons and background) are extracted: 1. a **spatial footprint** and 2. A **temporal trace**
- The *spatial footprint* describes the shape of each component whereas the *temporal trace* captures its fluorescence activity.
- Lastly, the neural activity of each fluorescent trace is deconvolved from the dynamics of the calcium indicator.
- These operations are challeging due to limited spatial resolution which results in spatially overlapping fluorescence trances of different sources and neuropil activity.

<font color='teal'> **Motion correction** </font> 

- `CaImAn` uses the `NoRMCorre` algorithm that corrects motion artifacts
- `NoRMCorre` algorithm stands for Non-Rigid Motion Correction.
- Motion is corrected by registering each frame of the imaging data to a reference frame. Unlike traditional rigid motion correction algorithms, which assume that all pixels move the same amount in the same direction, `NoRMCorre` allows for non-rigid transformations.

<font color='teal'> **Source extraction** </font> 

- Source extraction is performed using CNMF which can extract components (neurons and BG) with overlapping spatial footprints
- After motion correction, we can express each source as rank one matrices = outer(spatial footprint, temporal footprint)
- The full data as follows:

\begin{align}
Data = \sum_i (Rank \ 1 \ matrix)_i + Background + Neuropil + Noise
\end{align}

- `CaImAn Batch` embeds these approaches into a general framework with improved results in quality and speed versus the original CNMF.

<font color='teal'> **Deconvolution** </font> 

- Deconvolution is performed using sparse non-negative deconvolution and near-online OASIS

<font color='teal'> Online processing </font> 

<font color='teal'> **Batch processing of large scale datasets on standalone machines**</font> 

- The source extraction problem is inherently local with a neuron typically appearing in a neighbourhood within a small radius from its centre of mass.
- Exploiting this locality, `CaImAn Batch` splits the FOV into a set of **spatially overlapping** patches which enables the parallelisation of CNMF (for extracting the spatio-temporal and neuropil components)
- Processing in patches enables `CaImAn Batch` to detect neurons across the whole FOV.
- After all patches have been processed, the results are embedded within the FOV and the overlapping regions are processed such that components corresponding to the same neuron are merged.

<font color='teal'> Initialisation methods</font> 

<font color='teal'> **Automated component evaluation and classification**</font> 

- A limitation of matrix factorization algorithms is that the number of components must be pre-determined
- When processing large datasets in patches the target number of components is passed onto every patch implicitly assuming a uniform density of (active) neurons within the entire FOV. This assumption isn't generally true
- `CaImAn` introduces tests to eliminate false positives based on the pbservation that active components are bound to have a distinct localised spatio-temporal signature within the FOV.
- Such tests are: 

    1. Spatial footprint consistency: correlate the spatial footprint with the average frame of the data taken over intervals when the component was active. The component is rejected is the correaltion coefficient is bellow a threshold
    2. Trace SNR: for each component compute the peak SNR and reject if it's bellow threshold
    3. CNN based algorithm: CNN trained to classify spatil fottprints that resemble the soma of a neuron

<font color='teal'>Online analysis with CaImAn Online</font> 

<font color='teal'>**Component registration across multiple sessions** </font> 
- `CaImAn`uses an intersection over union metric to calculate the distance between different cells in different sessions
- To register the components between more than two sessions, they order the sessions chronologically and register the components of the current session against the union of the components of all past session aligned with the current FOV.

**RESULTS:** Read but information isn't relevant for this

<font color='teal'> **MATERIALS AND METHODS**</font>

<font color='teal'>**Memory mapping**</font> 

- Caiman Batch represents the datasets in a matrix form $Y$, where each **row corresponds to a different imaged pixel** and **each column to a different frame**.

<font color='teal'>**Mathematical model of the CNMF framework**</font> 

- The CNMF framework can be expressed as $Y = AC+B+E$.
    - $Y$ denotes the pbserved data in matrix form with dimensions $[d\times T]$. $d$ is the total number of observed pixels and $T$ the total number of observed time steps.
    - $A = [d \times N]$ denotes the matrix of the $N$ spatial footprints. $A = [\bm{a}_1, \bm{a}_2 ,...,\bm{a}_N]$ with $\bm{a}_i = [d \times 1]$ being the spatial component
    - $C = [N \times T]$ deontes the matrix of temporal components. $C = [\bm{c}_1, \bm{c}_2, ..., \bm{c}_N]^T$ with $\bm{c}_i = [T\times 1]$ being the temporal component
    - $B$ is the backkground/neuropil activity matrix. Its exact form depends if we are doing one- or two-photon imaging.

<font color='teal'>**Combining results from different patches**</font> 

- To combine results we need to account for the overlap at the boundaries.
- Neurons close to the boundary can appear multiple times and must be merged.
- Groups of components with spatially overlapping footprints whose temporal traces are correlated above a threshold are replace with a single component. 
- To comnie backgrounds from different patches, forst the backgrounds are approximated to a low rank matrix to obtain global spatial and temporal background components.
- Then all components and background from all the patches are combined.

<font color='teal'>**Initialization strategies**</font> 

- `GreedyROI`: first spatially smooths the data with a Gaussian kernel ($\sigma = \langle r_{neuron}\rangle$) and then initializes candidate components around location with maximum variance
- `RollingGreedyROI`: operates like `GreedyROI` but instead of working acroos all the data, it looks for points of maximum variance on a rolling window of fixed duration
- `GreedyCorr`: Initializes candidate components around locations that correspond to the local maxima of an image formed by the pointwise product between the correlation image and the peak SNR image. A threshold for acceptance of candidate neuron is used.
- `SeedInitialization`: Uses seed point annotations